# The Associated Press and Life Expectancy

**Story:** [AP analysis: Unemployment, income affect life expectancy](https://www.apnews.com/66ac44186b6249709501f07a7eab36da)

**Author:** Nicky Forster, Associated Press

**Topics:** Census Data, Linear Regression

**Datasets**

* **R12221544_SL140.csv:** ACS 2015 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table B23025: Employment Status
    - **R12221544.txt** is the data dictionary
* **R12221544_SL140.csv:** ACS 2015 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table B23025: Employment Status
    - Table B06009: Educational Attainment
    - Table B03002: Race
    - Table B19013: Median income
    - Table C17002: Ratio of income to poverty level
    - **R12221544.txt** is the data dictionary
* **US_A.CSV:** life expectancy by census tract, from [USALEEP](https://www.cdc.gov/nchs/nvss/usaleep/usaleep.html)
    - **Record_Layout_CensusTract_Life_Expectancy.pdf** is data dictionary

# What's the story?

We're trying to figure out how the **life expectancy in a census tract** is related to other factors like unemployment, income, and others.

# PREPWORK BONUS!

Download the data yourself from Social Explorer and USALEEP (linked above) instead of relying on the data included.

## Reading in our data

### Read in `USA_A.CSV`

Rename any columns with weird or not-understandable names as something more descriptive.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
df1 = pd.read_csv("US_A.CSV")

In [3]:
df1.columns

Index(['Tract ID', 'STATE2KX', 'CNTY2KX', 'TRACT2KX', 'e(0)', 'se(e(0))',
       'Abridged life table flag'],
      dtype='object')

In [4]:
df1 = df1.drop(columns=['Abridged life table flag','STATE2KX','CNTY2KX'])

In [5]:
df1 = df1.rename(columns={'TRACT2KX':'Census 2010 Tract',
'e(0)':'Life expectancy',
'se(e(0))':'Standard error of life expectancy at birth'})

In [6]:
df1.columns

Index(['Tract ID', 'Census 2010 Tract', 'Life expectancy',
       'Standard error of life expectancy at birth'],
      dtype='object')

### Open `R12221544_SL140.csv`

You'll need to give an option to `pd.read_csv` to make sure it's read in successfully.

In [7]:
df2 = pd.read_csv("R12221544_SL140.csv", encoding = 'latin1')

In [8]:
df2 = df2[['Geo_FIPS','Geo_TRACT','ACS15_5yr_B23025001',
       'ACS15_5yr_B23025002', 'ACS15_5yr_B23025003', 'ACS15_5yr_B23025004',
       'ACS15_5yr_B23025005', 'ACS15_5yr_B23025006', 'ACS15_5yr_B23025007']]

df2 = df2.rename(columns={'ACS15_5yr_B23025001':'Total',
       'ACS15_5yr_B23025002':'In Labor Force', 'ACS15_5yr_B23025003':'Civilian Labor Force', 'ACS15_5yr_B23025004':'Employed',
       'ACS15_5yr_B23025005':'Unemployed', 'ACS15_5yr_B23025006':'Armed Forces', 'ACS15_5yr_B23025007':'Not in Labour Force'}) 




In [9]:
df2.head()

,Geo_FIPS,Geo_TRACT,Total,In Labor Force,Civilian Labor Force,Employed,Unemployed,Armed Forces,Not in Labour Force
0,1001020100,20100,1554,997,997,943,54,0,557
1,1001020200,20200,1731,884,869,753,116,15,847
2,1001020300,20300,2462,1472,1464,1373,91,8,990
3,1001020400,20400,3424,2013,1998,1782,216,15,1411
4,1001020500,20500,8198,5461,5258,5037,221,203,2737


#### Filter out any columns we aren't interestd in

#### Create a new column for percent unemployment

We'll be using the total population in the census tract as the baseline for employment.

In [10]:
df2['Percent unemployed'] = df2.Unemployed / df2.Total*100

In [11]:
df2.head()

,Geo_FIPS,Geo_TRACT,Total,In Labor Force,Civilian Labor Force,Employed,Unemployed,Armed Forces,Not in Labour Force,Percent unemployed
0,1001020100,20100,1554,997,997,943,54,0,557,3.474903
1,1001020200,20200,1731,884,869,753,116,15,847,6.701329
2,1001020300,20300,2462,1472,1464,1373,91,8,990,3.696182
3,1001020400,20400,3424,2013,1998,1782,216,15,1411,6.308411
4,1001020500,20500,8198,5461,5258,5037,221,203,2737,2.695779


## Merging the data

Merge the dataframes together based on their census tract.

In [12]:
merged = df1.merge(df2, left_on='Census 2010 Tract',right_on='Geo_TRACT')

In [13]:
merged.head()

,Tract ID,Census 2010 Tract,Life expectancy,Standard error of life expectancy at birth,Geo_FIPS,Geo_TRACT,Total,In Labor Force,Civilian Labor Force,Employed,Unemployed,Armed Forces,Not in Labour Force,Percent unemployed
0,1001020100,20100,73.1,2.2348,1001020100,20100,1554,997,997,943,54,0,557,3.474903
1,1001020100,20100,73.1,2.2348,1033020100,20100,2972,1467,1467,1328,139,0,1505,4.676985
2,1001020100,20100,73.1,2.2348,1045020100,20100,2791,1432,1432,1320,112,0,1359,4.012899
3,1001020100,20100,73.1,2.2348,1057020100,20100,2601,1373,1373,1279,94,0,1228,3.613995
4,1001020100,20100,73.1,2.2348,1127020100,20100,2680,1409,1409,1200,209,0,1271,7.798507


## Running the regression

Using the `statsmodels` package, run a linear regression to find the coefficient relating unemployment and life expectancy.

In [15]:
X = merged['Percent unemployed']
y = merged['Life expectancy']
mod = sm.OLS(y, X, missing='drop')
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:        Life expectancy   R-squared (uncentered):                   0.729
Model:                            OLS   Adj. R-squared (uncentered):              0.729
Method:                 Least Squares   F-statistic:                          7.342e+06
Date:                Tue, 16 Jul 2019   Prob (F-statistic):                        0.00
Time:                        16:54:49   Log-Likelihood:                     -1.3948e+07
No. Observations:             2728570   AIC:                                  2.790e+07
Df Residuals:                 2728569   BIC:                                  2.790e+07
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Percent unemployed    10.3464      0.004   2709.609      0.000      10.339      10.354
==============================================================================
Omnibus:                   744981.767   Durbin-Watson:                   1.288
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2349730.257
Skew:                          -1.400   Prob(JB):                         0.00
Kurtosis:                       6.582   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# X = sm.add_constant(merged[['Percent unemployed']])
X = sm.add_constant(merged[['Percent unemployed']])
y = merged['Life expectancy']
mod = sm.OLS(Y, X, missing='drop')
res = mod.fit()
res.summary()

Translate that into the form **"every X percentage point change in unemployment translates to a Y change in life expectancy"**

In [ ]:
# For every percentage increase in unemployment, life expectancy decreases by 0.0365 years

## Bringing more columns into the mix

Only dealing with unemployment seems kind of narrow-minded, let's expand our reach a bit.

### Read in `R12221544_SL140.csv`

It's also from the Census, and has many, many, many more columns available to you compared to the list dataset.

Using this census data, create a new dataframe that includes the following columns:

* Percent unemployed
* Percents Black, White, and Hispanic
* Median Income (in increments of 10,000 dollars)
* Percent of the population with less than a high school education
* Percent of the population between 1-1.5x the poverty line

If you have to many any editorial decisions about which columns you choose or how you do your math, please explain them.

In [16]:
pd.set_option('display.max_columns', None)

In [17]:
df3 = pd.read_csv("R12221550_SL140.csv", encoding='latin1')

In [19]:
df4 = df3[['Geo_TRACT',
           'Geo_FIPS',
           'ACS15_5yr_B23025005',
           'ACS15_5yr_B23025001',
           'ACS15_5yr_B03002003',
           'ACS15_5yr_B03002004',
           'ACS15_5yr_B03002012',
           'ACS15_5yr_B03002001',
           'ACS15_5yr_B19013001',
           'ACS15_5yr_B06009002',
           'ACS15_5yr_B06009001',
           'ACS15_5yr_C17002001',
           'ACS15_5yr_C17002004',
           'ACS15_5yr_C17002005']]

In [20]:
df4 = df4.rename(columns={'ACS15_5yr_B23025005':'Unemployed',
 'ACS15_5yr_B23025001':'Employed_total',
 'ACS15_5yr_B03002003':'White',
 'ACS15_5yr_B03002004':'Black',
 'ACS15_5yr_B03002012':'Hispanic',
 'ACS15_5yr_B03002001':'Race_total',
 'ACS15_5yr_B19013001':'Median income',
 'ACS15_5yr_B06009002':'Education_less_than_high_school',
 'ACS15_5yr_B06009001':'Education_total',
 'ACS15_5yr_C17002001':'Poverty_total',
 'ACS15_5yr_C17002004':'Poverty Status 1.00 to 1.24',
 'ACS15_5yr_C17002005':'Poverty Status 1.25 to 1.49'})

df4.head()

,Geo_TRACT,Geo_FIPS,Unemployed,Employed_total,White,Black,Hispanic,Race_total,Median income,Education_less_than_high_school,Education_total,Poverty_total,Poverty Status 1.00 to 1.24,Poverty Status 1.25 to 1.49
0,20100,1001020100,54,1554,1703,150,17,1948,61838.0,184.0,1243.0,1948,81,101
1,20200,1001020200,116,1731,872,1149,17,2156,32303.0,356.0,1397.0,1983,232,58
2,20300,1001020300,91,2462,2212,551,0,2968,44922.0,221.0,2074.0,2968,148,207
3,20400,1001020400,216,3424,3662,162,464,4423,54329.0,339.0,2899.0,4423,141,182
4,20500,1001020500,221,8198,7368,2674,80,10763,51965.0,310.0,6974.0,10563,256,1064


In [21]:
df4['Percent_unemployed']=df4.Unemployed/df4.Employed_total*100

In [22]:
df4['Percent_white']=df4.White/df4.Race_total*100
df4['Percent_black']=df4.Black/df4.Race_total*100
df4['Percent_hispanic']=df4.Hispanic/df4.Race_total*100

In [23]:
df4['Percent_highschool']=df4.Education_less_than_high_school/df4.Education_total*100

In [24]:
df4['Percent_poverty_1_to_1.24'] = df4['Poverty Status 1.00 to 1.24']/df4.Poverty_total*100

In [25]:
df4['Percent_poverty_1.24_to_1.49'] = df4['Poverty Status 1.25 to 1.49']/df4.Poverty_total*100

In [28]:
df5 = df4[['Geo_TRACT','Geo_FIPS','Percent_unemployed','Percent_white','Percent_black','Percent_hispanic','Percent_highschool','Percent_poverty_1_to_1.24','Percent_poverty_1.24_to_1.49','Median income']]

In [29]:
df5.head()

,Geo_TRACT,Geo_FIPS,Percent_unemployed,Percent_white,Percent_black,Percent_hispanic,Percent_highschool,Percent_poverty_1_to_1.24,Percent_poverty_1.24_to_1.49,Median income
0,20100,1001020100,3.474903,87.422998,7.700205,0.872690,14.802896,4.158111,5.184805,61838.0
1,20200,1001020200,6.701329,40.445269,53.293135,0.788497,25.483178,11.699445,2.924861,32303.0
2,20300,1001020300,3.696182,74.528302,18.564690,0.000000,10.655738,4.986523,6.974394,44922.0
3,20400,1001020400,6.308411,82.794483,3.662672,10.490617,11.693687,3.187882,4.114854,54329.0
4,20500,1001020500,2.695779,68.456750,24.844374,0.743287,4.445082,2.423554,10.072896,51965.0


### Join your datasets

Combine your life expectancy dataset with this census dataset to create a new dataframe.

In [30]:
final_df = merged.merge(df5, left_on='Geo_FIPS', right_on='Geo_FIPS')

In [32]:
final_df.head(30)

,Tract ID,Census 2010 Tract,Life expectancy,Standard error of life expectancy at birth,Geo_FIPS,Geo_TRACT_x,Total,In Labor Force,Civilian Labor Force,Employed,Unemployed,Armed Forces,Not in Labour Force,Percent unemployed,Geo_TRACT_y,Percent_unemployed,Percent_white,Percent_black,Percent_hispanic,Percent_highschool,Percent_poverty_1_to_1.24,Percent_poverty_1.24_to_1.49,Median income
0,1001020100,20100,73.1,2.2348,1001020100,20100,1554,997,997,943,54,0,557,3.474903,20100,3.474903,87.422998,7.700205,0.87269,14.802896,4.158111,5.184805,61838.0
1,1033020100,20100,72.3,1.4305,1001020100,20100,1554,997,997,943,54,0,557,3.474903,20100,3.474903,87.422998,7.700205,0.87269,14.802896,4.158111,5.184805,61838.0
2,1045020100,20100,75.0,1.5468,1001020100,20100,1554,997,997,943,54,0,557,3.474903,20100,3.474903,87.422998,7.700205,0.87269,14.802896,4.158111,5.184805,61838.0
3,1057020100,20100,72.6,1.4231,1001020100,20100,1554,997,997,943,54,0,557,3.474903,20100,3.474903,87.422998,7.700205,0.87269,14.802896,4.158111,5.184805,61838.0
4,1127020100,20100,68.6,1.3919,1001020100,20100,1554,997,997,943,54,0,557,3.474903,20100,3.474903,87.422998,7.700205,0.87269,14.802896,4.158111,5.184805,61838.0
5,4012020100,20100,78.9,1.3106,1001020100,20100,1554,997,997,943,54,0,557,3.474903,20100,3.474903,87.422998,7.700205,0.87269,14.802896,4.158111,5.184805,61838.0
6,5033020100,20100,76.4,1.8804,1001020100,20100,1554,997,997,943,54,0,557,3.474903,20100,3.474903,87.422998,7.700205,0.87269,14.802896,4.158111,5.184805,61838.0
7,5059020100,20100,73.4,1.2765,1001020100,20100,1554,997,997,943,54,0,557,3.474903,20100,3.474903,87.422998,7.700205,0.87269,14.802896,4.158111,5.184805,61838.0
8,5091020100,20100,72.9,1.3771,1001020100,20100,1554,997,997,943,54,0,557,3.474903,20100,3.474903,87.422998,7.700205,0.87269,14.802896,4.158111,5.184805,61838.0
9,6075020100,20100,75.7,2.0606,1001020100,20100,1554,997,997,943,54,0,557,3.474903,20100,3.474903,87.422998,7.700205,0.87269,14.802896,4.158111,5.184805,61838.0


## Running your multivariate regression

Using the `statsmodels` package and this new dataframe, run a multivariate linear regression to find the coefficient relating your columns and life expectancy.

In [37]:
X = sm.add_constant(final_df[['Percent unemployed', 'Percent_white','Percent_black','Percent_hispanic','Percent_highschool','Percent_poverty_1_to_1.24','Percent_poverty_1.24_to_1.49','Median income']])
y = final_df['Life expectancy']
mod = sm.OLS(y, X, missing='drop')
res = mod.fit()
res.summary()

/Users/amyokruk/.pyenv/versions/3.6.5/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life expectancy   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1188.
Date:                Tue, 16 Jul 2019   Prob (F-statistic):               0.00
Time:                        17:10:19   Log-Likelihood:            -7.6933e+06
No. Observations:             2708347   AIC:                         1.539e+07
Df Residuals:                 2708338   BIC:                         1.539e+07
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                           76.4344      0.034   2243.017      0.000      76.368      76.501
Percent unemployed              -0.0100      0.001    -10.430      0.000      -0.012      -0.008
Percent_white                    0.0007      0.000      2.281      0.023    9.82e-05       0.001
Percent_black                    0.0001      0.000      0.452      0.651      -0.000       0.001
Percent_hispanic              7.869e-05      0.000      0.225      0.822      -0.001       0.001
Percent_highschool               0.0063      0.000     16.329      0.000       0.006       0.007
Percent_poverty_1_to_1.24       -0.0060      0.001     -7.361      0.000      -0.008      -0.004
Percent_poverty_1.24_to_1.49    -0.0004      0.001     -0.538      0.591      -0.002       0.001
Median income                 1.196e-05   1.73e-07     69.020      0.000    1.16e-05    1.23e-05
==============================================================================
Omnibus:                     8864.754   Durbin-Watson:                   1.717
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10368.017
Skew:                          -0.084   Prob(JB):                         0.00
Kurtosis:                       3.252   Cond. No.                     6.77e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.77e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Translate some of your coefficients into the form **"every X percentage point change in unemployment translates to a Y change in life expectancy."** Do this with numbers that are meaningful, and in a way that is easily understandable to your reader.